In [47]:
import numpy as np 
import pandas as pd 
import io
import os
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from imageio import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data
# json for storing data in json file
import json

In [48]:
import os
os.chdir('C:/Users/rishv/OneDrive/Northeastern/SEM3/Algorithmic Digital Marketing/Assignments/Assignment3/Dataset')
images_dir = os.path.join('C:/Users/rishv/OneDrive/Northeastern/SEM3/Algorithmic Digital Marketing/Assignments/Assignment3/Dataset', 'image')

In [49]:
data = bson.decode_file_iter(open('C:/Users/rishv/OneDrive/Northeastern/SEM3/Algorithmic Digital Marketing/Assignments/Assignment3/Dataset/train.bson', 'rb'))

product_count=1

#prod_to_category = dict()
prod_count = dict()
prod_count_3 = dict()

for c, d in enumerate(data):
    product_id = d['_id']
    category_id = d['category_id'] # This won't be in Test data
    #print("values",category_id, product_id)
    if(category_id in prod_count) :
        #print("yes")
        product_count = prod_count[category_id]+1
        prod_count[category_id] = product_count
        #print("prod_count",prod_count)
        if(product_count==100):
            prod_count_3[category_id] = product_count
            #print("prod_count_3",prod_count_3)
    else:
        #print(category_id, product_count)
        product_count=1
        prod_count[category_id] = product_count
        #print("prod_count",prod_count)
    
    if(len(prod_count_3)==100):
        break
        
#print("product",prod_count_3)
        

In [53]:
data = bson.decode_file_iter(open('C:/Users/rishv/OneDrive/Northeastern/SEM3/Algorithmic Digital Marketing/Assignments/Assignment3/Dataset/train.bson', 'rb'))

image=[]

prod_to_category = dict()
cat_prod_count = dict()
product_count=1
for c, d in enumerate(data):
    product_id = d['_id']
    category_id = d['category_id'] # This won't be in Test data
    if(category_id in prod_count_3):
        if(category_id in cat_prod_count):
            if(cat_prod_count[category_id]<=100):
                product_count = cat_prod_count[category_id]+1
                cat_prod_count[category_id]=product_count
                prod_to_category[product_id] = category_id
                for e, pic in enumerate(d['imgs']):
                    picture_count+=1
                    picture = imread(io.BytesIO(pic['picture']))
                    picture_file = os.path.join(images_dir, str(product_id) + '_' + str(e) + '.jpg')
                    plt.imsave(picture_file, picture)
                    
                    image.append({
                    'ImageName':str(product_id) + '_' + str(e),
                    'ProductId': product_id,
                    'CategoryId': category_id})
                    
                                
            else:
                continue
        else:
            product_count=1
            cat_prod_count[category_id]=product_count
            #print(cat_prod_count)
        
    else:
        #print("no")
        continue
    #if(prod_to_category.shape[0]==10000):
        #break
        
with open('image.json', 'w') as out:
        json.dump(image, out)

prod_to_category = pd.DataFrame.from_dict(prod_to_category, orient='index')
prod_to_category.index.name = '_id'
prod_to_category.rename(columns={0: 'category_id'}, inplace=True)

In [54]:
prod_to_category.shape

(10000, 1)

In [52]:
prod_to_category.to_csv('prod_to_category.csv') 